In [1]:
# place all the points on the map of last-seen-nonstat, excluding itself
# folds information actually not necessary
import os
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
print(os.listdir("../input/eyn-original"))
x_min, x_max = -1., 1.
y_min, y_max = -.3, .3

QUERY_SIZE = 10

['train_data.npy', 'trn_index_list.npy', 'custom.css', 'val_index_list.npy', '__results__.html', 'train_targets_inside.npy', 'submission.csv', 'train_targets.npy', 'test_ids.npy', '__output__.json', 'test_data.npy', '__notebook__.ipynb']


In [2]:
train_data = np.load("../input/eyn-original/train_data.npy")
test_data = np.load("../input/eyn-original/test_data.npy")
train_targets = np.load("../input/eyn-original/train_targets.npy")
train_targets_inside = np.load("../input/eyn-original/train_targets_inside.npy")
train_targets_inside_indexes = np.argwhere(train_targets_inside == 1)
print(train_data.shape, train_targets.shape)
print(train_targets_inside.shape, train_targets_inside_indexes.shape)
print(test_data.shape)
# 't_entry','t_exit','x_entry','y_entry','x_exit','y_exit','vmax','vmin','vmean','tid_0','tid_1'

(134063, 21, 11) (134063, 2)
(134063,) (36380, 1)
(33515, 21, 11)


In [3]:
# the queries
train_entry_loc = np.concatenate((train_data[:,:,2:4], 
                                  train_data[:,:,0:2]*0.0001 + 
                                  np.random.randn(*train_data[:,:,0:2].shape)*0.000001), axis=2)
train_exit_loc = np.concatenate((train_data[:,:,4:6], train_data[:,:,0:2]*0.0001), axis=2)
test_entry_loc = np.concatenate((test_data[:,:,2:4], test_data[:,:,0:2]*0.0001), axis=2)
test_exit_loc = np.concatenate((test_data[:,:,4:6], test_data[:,:,0:2]*0.0001), axis=2)
print(train_entry_loc.shape, train_exit_loc.shape)
print(test_entry_loc.shape, test_exit_loc.shape)

(134063, 21, 4) (134063, 21, 4)
(33515, 21, 4) (33515, 21, 4)


In [4]:
# building the reference
train_last_nonstat_indexes = np.load("../input/eyn-folds/train_last_not_stationary.npy")
print(train_last_nonstat_indexes.shape)
train_last_nonstat_loc = train_entry_loc[train_last_nonstat_indexes, 0, :] 
train_last_nonstat_target = train_targets_inside[train_last_nonstat_indexes]
print(train_last_nonstat_loc.shape, train_last_nonstat_target.shape)

(67065,)
(67065, 4) (67065,)


In [5]:
from scipy import spatial

reference = train_last_nonstat_loc
reference_tree = spatial.KDTree(reference)

In [6]:
train_entry_query_dist, train_entry_query_indexes = reference_tree.query(train_entry_loc,k=QUERY_SIZE)
print(train_entry_query_dist.shape, train_entry_query_indexes.shape)

(134063, 21, 10) (134063, 21, 10)


In [7]:
train_exit_query_dist, train_exit_query_indexes = reference_tree.query(train_exit_loc,k=QUERY_SIZE)
print(train_exit_query_dist.shape, train_exit_query_indexes.shape)

(134063, 21, 10) (134063, 21, 10)


In [8]:
test_entry_query_dist, test_entry_query_indexes = reference_tree.query(test_entry_loc,k=QUERY_SIZE)
print(test_entry_query_dist.shape, test_entry_query_indexes.shape)

(33515, 21, 10) (33515, 21, 10)


In [9]:
test_exit_query_dist, test_exit_query_indexes = reference_tree.query(test_exit_loc,k=QUERY_SIZE)
print(test_exit_query_dist.shape, test_exit_query_indexes.shape)

(33515, 21, 10) (33515, 21, 10)


In [10]:
train_entry_query_target = np.take(train_last_nonstat_target, train_entry_query_indexes, mode = 'clip')
train_exit_query_target = np.take(train_last_nonstat_target, train_exit_query_indexes, mode = 'clip')
test_entry_query_target = np.take(train_last_nonstat_target, test_entry_query_indexes, mode = 'clip')
test_exit_query_target = np.take(train_last_nonstat_target, test_exit_query_indexes, mode = 'clip')
print(train_entry_query_target.shape, train_exit_query_target.shape)
print(test_entry_query_target.shape, test_exit_query_target.shape)

(134063, 21, 10) (134063, 21, 10)
(33515, 21, 10) (33515, 21, 10)


In [11]:
# final exit values should be nans
train_exit_query_target[:,0,:] = np.nan
train_exit_query_dist[:,0,:] = np.nan
test_exit_query_target[:,0,:] = np.nan
test_exit_query_dist[:,0,:] = np.nan

# should exclude the last-seen-position
train_entry_query_target[:,0,0] = np.nan
train_entry_query_dist[:,0,0] = np.nan

In [12]:
print(train_entry_query_target.shape, train_exit_query_target.shape)
print(train_entry_query_dist.shape, train_exit_query_dist.shape)
print(test_entry_query_target.shape, test_exit_query_target.shape)
print(test_entry_query_dist.shape, test_exit_query_dist.shape)

(134063, 21, 10) (134063, 21, 10)
(134063, 21, 10) (134063, 21, 10)
(33515, 21, 10) (33515, 21, 10)
(33515, 21, 10) (33515, 21, 10)


In [13]:
df_train = pd.read_pickle("../input/eyn-pre-unravel-df/df_train.pickle")
df_test = pd.read_pickle("../input/eyn-pre-unravel-df/df_test.pickle")
df_original_columns = df_train.columns

In [14]:
entry_col_names_target = ["at{}".format(i) for i in range(QUERY_SIZE)]
entry_col_names_dist = ["ad{}".format(i) for i in range(QUERY_SIZE)]
exit_col_names_target = ["bt{}".format(i) for i in range(QUERY_SIZE)]
exit_col_names_dist = ["bd{}".format(i) for i in range(QUERY_SIZE)]

In [15]:
for i,(a,b,c,d) in enumerate(zip(entry_col_names_target, exit_col_names_target,
                                 entry_col_names_dist, exit_col_names_dist)):
    df_train[a] = train_entry_query_target[:,:,i].ravel()
    df_train[c] = train_entry_query_dist[:,:,i].ravel()
    df_train[b] = train_exit_query_target[:,:,i].ravel()
    df_train[d] = train_exit_query_dist[:,:,i].ravel()
    
    df_test[a] = test_entry_query_target[:,:,i].ravel()
    df_test[c] = test_entry_query_dist[:,:,i].ravel()
    df_test[b] = test_exit_query_target[:,:,i].ravel()
    df_test[d] = test_exit_query_dist[:,:,i].ravel()

In [16]:
# reinforce nan values for nan entries
df_train.loc[np.isnan(train_data[:,:,2].ravel()), :] = np.nan
df_test.loc[np.isnan(test_data[:,:,2].ravel()), :] = np.nan

In [17]:
df_train.head(8)

t_entry    t_exit   x_entry    ...          ad9  bt9       bd9
seq traj                                  ...                            
0   0     0.004194  0.030917 -1.595688    ...     0.004642  NaN       NaN
    1    -0.036417 -0.036417 -1.599250    ...     0.002164  0.0  0.002164
    2    -0.670278 -0.637694 -1.602144    ...     0.000336  0.0  0.008035
    3    -0.710778 -0.694306 -1.603369    ...     0.002303  0.0  0.002017
    4    -0.765722 -0.757167 -1.696432    ...     0.032572  0.0  0.005448
    5    -0.792472 -0.785778 -0.988774    ...     0.011113  0.0  0.023875
    6          NaN       NaN       NaN    ...          NaN  NaN       NaN
    7          NaN       NaN       NaN    ...          NaN  NaN       NaN

[8 rows x 63 columns]

In [18]:
df_test.head(6)

t_entry    t_exit   x_entry    ...          ad9  bt9       bd9
seq traj                                  ...                            
0   0     0.005889  0.017556  1.221697    ...     0.019193  NaN       NaN
    1    -0.157417 -0.127972  1.248303    ...     0.011843  0.0  0.016829
    2    -0.242583 -0.176361  0.285834    ...     0.008330  0.0  0.016480
    3    -0.263972 -0.263972  1.229712    ...     0.011442  0.0  0.011442
    4    -0.327861 -0.316194  1.251197    ...     0.011159  0.0  0.020502
    5          NaN       NaN       NaN    ...          NaN  NaN       NaN

[6 rows x 63 columns]

In [19]:
df_train[entry_col_names_target] = df_train[entry_col_names_target].astype('category')
df_train[exit_col_names_target] = df_train[exit_col_names_target].astype('category')
df_test[entry_col_names_target] = df_test[entry_col_names_target].astype('category')
df_test[exit_col_names_target] = df_test[exit_col_names_target].astype('category')
df_train = df_train.drop(columns=df_original_columns)
df_test = df_test.drop(columns=df_original_columns)

In [20]:
df_train.head(10)

at0       ad0  bt0       bd0    ...     at9       ad9  bt9       bd9
seq traj                                  ...                                 
0   0     NaN       NaN  NaN       NaN    ...     0.0  0.004642  NaN       NaN
    1     0.0  0.000724  0.0  0.000724    ...     0.0  0.002164  0.0  0.002164
    2     0.0  0.000145  0.0  0.002898    ...     0.0  0.000336  0.0  0.008035
    3     0.0  0.001014  0.0  0.000649    ...     0.0  0.002303  0.0  0.002017
    4     0.0  0.016330  0.0  0.000426    ...     0.0  0.032572  0.0  0.005448
    5     0.0  0.001168  0.0  0.004895    ...     0.0  0.011113  0.0  0.023875
    6     NaN       NaN  NaN       NaN    ...     NaN       NaN  NaN       NaN
    7     NaN       NaN  NaN       NaN    ...     NaN       NaN  NaN       NaN
    8     NaN       NaN  NaN       NaN    ...     NaN       NaN  NaN       NaN
    9     NaN       NaN  NaN       NaN    ...     NaN       NaN  NaN       NaN

[10 rows x 40 columns]

In [21]:
df_train.to_pickle("df_train_cluster.pickle")
df_test.to_pickle("df_test_cluster.pickle")

In [22]:
# to load
df_train = pd.read_pickle("df_train_cluster.pickle")
df_test = pd.read_pickle("df_test_cluster.pickle")
print(df_train.shape, df_test.shape)

(2815323, 40) (703815, 40)


In [23]:
str_list = []
for i in range(len(train_entry_query_dist)):
    str_list.append(str([train_entry_query_dist[i,0,:]]))
print(len(str_list))
print(len(set(str_list)))
print(len(np.array(str_list)[train_last_nonstat_indexes]))
print(len(set(np.array(str_list)[train_last_nonstat_indexes])))

134063
133877
67065
67065


In [24]:
a = np.argwhere((train_entry_query_indexes[:,0,0][train_last_nonstat_indexes] - np.arange(len(train_last_nonstat_indexes)))!=0)[:,0]
a

array([], dtype=int64)

In [25]:
train_entry_query_indexes[:,0,0][train_last_nonstat_indexes][a]

array([], dtype=int64)

In [26]:
train_entry_query_indexes[:,0,1][train_last_nonstat_indexes][a]

array([], dtype=int64)

In [27]:
train_entry_query_indexes[:,0,2][train_last_nonstat_indexes][a]

array([], dtype=int64)